Creating an etl Job

In [3]:
from spark_session_factory import create_spark_session
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window


In [55]:
def run_etl(spark: SparkSession, input_path_topic1: str, input_path_topic2: str, output_path: str):
    """
    Main ETL pipeline: read -> transform -> write.
    
    Args:
        spark: Active SparkSession
        input_path: Landing zone path (e.g., '/opt/spark-data/landing/*.json')
        output_path: Gold zone path (e.g., '/opt/spark-data/gold')
    """
    # TODO: Implement
    spark = spark
    
    df_topic_transaction = spark.read.json(input_path_topic1).cache()
    df_topic_user = spark.read.json(input_path_topic2).cache()
    
    # df_topic_transaction.show(truncate=False)
    # df_topic_user.show(truncate=False)
    
    # print(df_topic_transaction.printSchema())
    # print(df_topic_user.printSchema())
    
    df_exploded_transaction = df_topic_transaction.withColumn("product", F.explode("products"))
    df_exploded_transaction.select("product.product_id").show(truncate= False)
    
    print(df_exploded_transaction.printSchema())
    # print(df_topic1.select("*").filter(F.col("user_id") == '10f1bc81').count())
    # print(df_topic2.select('*').filter(F.col("user_id") == '10f1bc81').count())
    
    # Join on user_id and the product_id being the same
    df_topics_combined = df_exploded_transaction.alias("transaction") \
        .join(df_topic_user.alias("user"), 
              ((F.col("user.user_id") == F.col("transaction.user_id")) & (F.col("user.product_id") == F.col("transaction.product.product_id"))), 
              "inner") 
    
    print(df_topic_user.count())
    print(df_topic_transaction.count())
    print(df_topics_combined.show())
    

    # topics_combined.select('*').filter(F.col("user_id") == '10f1bc81').show(100,truncate=False)

    
    spark.stop()
    pass


Questions we want to answer!

We can join on by users,
We can rank devices most likely to spend the most

In [56]:
run_etl(create_spark_session("ETL_job"),'../data/landing/transaction_events_1767986916.1293132.json', '../data/landing/user_events_1767986960.934074.json', "")

+----------+
|product_id|
+----------+
|PROD_1078 |
|PROD_1069 |
|PROD_1012 |
|PROD_1140 |
|PROD_1082 |
|PROD_1060 |
|PROD_1024 |
|PROD_1181 |
|PROD_1105 |
|PROD_1082 |
|PROD_1045 |
|PROD_1157 |
|PROD_1138 |
|PROD_1140 |
|PROD_1089 |
|PROD_1161 |
|PROD_1082 |
|PROD_1130 |
|PROD_1041 |
|PROD_1180 |
+----------+
only showing top 20 rows

root
 |-- billing_address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- street: string (nullable = true)
 |    |-- zip_code: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- original_transaction_id: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- product_id: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    